In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"  # specify which GPU(s) to be used

import sys
sys.argv = ['GPT_eval_multi.py']
import options.option_transformer as option_trans
args = option_trans.get_args_parser()

args.dataname = 't2m'
args.resume_pth = 'pretrained/VQVAE/net_last.pth'
args.resume_trans = 'pretrained/VQTransformer_corruption05/net_best_fid.pth'
args.down_t = 2
args.depth = 3
args.block_size = 51
import clip
import torch
import numpy as np
import models.vqvae as vqvae
import models.t2m_trans as trans
import warnings
warnings.filterwarnings('ignore')

from utils.motion_process import recover_from_ric, recover_from_rot


In [3]:
from utils.skeleton import Skeleton
from utils.paramUtil import *
import visualization.plot_3d_global as plot_3d

skeleton = Skeleton(offset=torch.from_numpy(t2m_raw_offsets), 
                    kinematic_tree=t2m_kinematic_chain,
                    device='cuda')
skeleton.set_offset(torch.from_numpy(t2m_raw_offsets))

In [8]:
import codecs as cs

split_file = os.path.join('dataset/HumanML3D/val.txt')
joints_num = 22

data_dict = {}
id_list = []
with cs.open(split_file, 'r') as f:
    for line in f.readlines():
        id_list.append(line.strip())


mean = torch.from_numpy(np.load('./checkpoints/t2m/VQVAEV3_CB1024_CMT_H1024_NRES3/meta/mean.npy')).cuda()
std = torch.from_numpy(np.load('./checkpoints/t2m/VQVAEV3_CB1024_CMT_H1024_NRES3/meta/std.npy')).cuda()

In [4]:
net = vqvae.HumanVQVAE(args, ## use args to define different parameters in different quantizers
                       args.nb_code,
                       args.code_dim,
                       args.output_emb_width,
                       args.down_t,
                       args.stride_t,
                       args.width,
                       args.depth,
                       args.dilation_growth_rate)

print ('loading checkpoint from {}'.format(args.resume_pth))
ckpt = torch.load(args.resume_pth, map_location='cpu')
net.load_state_dict(ckpt['net'], strict=True)
net.eval()
net.cuda()

loading checkpoint from pretrained/VQVAE/net_last.pth


HumanVQVAE(
  (vqvae): VQVAE_251(
    (encoder): Encoder(
      (model): Sequential(
        (0): Conv1d(263, 512, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU()
        (2): Sequential(
          (0): Conv1d(512, 512, kernel_size=(4,), stride=(2,), padding=(1,))
          (1): Resnet1D(
            (model): Sequential(
              (0): ResConv1DBlock(
                (norm1): Identity()
                (norm2): Identity()
                (activation1): ReLU()
                (activation2): ReLU()
                (conv1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(9,), dilation=(9,))
                (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
              )
              (1): ResConv1DBlock(
                (norm1): Identity()
                (norm2): Identity()
                (activation1): ReLU()
                (activation2): ReLU()
                (conv1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
 

In [27]:
motion = np.load(os.path.join('dataset/HumanML3D/new_joint_vecs', id_list[37] + '.npy'))

motion = torch.from_numpy(motion).cuda()
motion = (motion - mean) / std

motion = motion[None, ...]
# motion = motion.cuda()

In [28]:
pred_pose, loss_commit, perplexity = net(motion[0:0+1, :])
pred_pose = pred_pose.detach()

In [29]:
xyz = recover_from_rot((motion*std+mean).float(), 22, skeleton)
xyz = xyz.reshape(1, -1, 22, 3)
xyz = xyz / 6.0

In [30]:
pose_xyz = recover_from_rot((pred_pose*std+mean).float(), 22, skeleton)
pose_xyz = pose_xyz.reshape(1, -1, 22, 3)
pose_xyz = pose_xyz / 6.0

In [31]:
import visualization.plot_3d_global as plot_3d
pose_vis = plot_3d.draw_to_batch(pose_xyz.detach().cpu().numpy(), 'debug', ['./results/debug_recover_from_rot/3.gif'])
pose_vis = plot_3d.draw_to_batch(xyz.detach().cpu().numpy(), 'debug', ['./results/debug_recover_from_rot/3_gt.gif'])